In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from utils.misc import MetricLogger

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
from utils.arguments import load_vcoco_opt_command, load_vcoco_parser

cmdline_args = load_vcoco_parser()
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]

model_path = '../data/output/test/00232200_best/default/raw_model_states.pt'
cmdline_args.overrides = ['DONT_LOAD_MODEL', 'false', 'PYLEARN_MODEL', model_path] 

opt = load_vcoco_opt_command(cmdline_args)
opt = init_distributed(opt)

In [4]:
print(opt['base_path'])
print(opt["RESUME"])

../
False


In [5]:
# opt["POSTPROCESS"]["OFFICIAL"]["USE"] = True

In [6]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-----------------------------------------------
INFO:trainer.default_trainer:Evaluating model ... 
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (bloc

Evaluation start ...


INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/2473. Dataloading: 0.0145 s/iter. Inference: 0.1153 s/iter. Eval: 0.0002 s/iter. Total: 0.1300 s/iter. ETA=0:05:20
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 50/2473. Dataloading: 0.0169 s/iter. Inference: 0.1136 s/iter. Eval: 0.0002 s/iter. Total: 0.1307 s/iter. ETA=0:05:16
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 89/2473. Dataloading: 0.0168 s/iter. Inference: 0.1125 s/iter. Eval: 0.0002 s/iter. Total: 0.1296 s/iter. ETA=0:05:08
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 128/2473. Dataloading: 0.0168 s/iter. Inference: 0.1121 s/iter. Eval: 0.0002 s/iter. Total: 0.1292 s/iter. ETA=0:05:02
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 168/2473. Dataloading: 0.0168 s/iter. Inference: 0.1115 s/iter. Eval: 0.0002 s/iter. Total: 0.1286 s/iter. ETA=0:04:56
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 207/2473. Dataloading: 0.0168 s/iter. Inference: 0.1114 s/iter. Eval: 0.0002 s

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.3990
                  stand: #GTs = 4118, AP = 0.7750
              sit_instr: #GTs = 1916, AP = 0.4238
             ride_instr: #GTs = 0556, AP = 0.5330
                   walk: #GTs = 0597, AP = 0.5058
               look_obj: #GTs = 3347, AP = 0.3732
              hit_instr: #GTs = 0349, AP = 0.5901
                hit_obj: #GTs = 0349, AP = 0.2891
                eat_obj: #GTs = 0521, AP = 0.4578
              eat_instr: #GTs = 0521, AP = 0.6554
             jump_instr: #GTs = 0635, AP = 0.6457
              lay_instr: #GTs = 0387, AP = 0.4851
    talk_on_phone_instr: #GTs = 0285, AP = 0.3253
              carry_obj: #GTs = 0472, AP = 0.3106
              throw_obj: #GTs = 0244, AP = 0.3797
              catch_obj: #GTs = 0246, AP = 0.3983
              cut_instr: #GTs = 0269, AP = 0.2928
                cut_obj: #GTs = 0269, AP = 0.5724
                    run: #GTs = 0687, A

INFO:trainer.default_trainer:{'vcoco_val/vcoco': {'AP_hold_obj': 0.3989988749101249, 'AP_stand': 0.7750321864972292, 'AP_sit_instr': 0.423760818229507, 'AP_ride_instr': 0.5329947832430317, 'AP_walk': 0.5057719283058536, 'AP_look_obj': 0.37318816204639255, 'AP_hit_instr': 0.5900986007457361, 'AP_hit_obj': 0.28914454117954863, 'AP_eat_obj': 0.4578180872922468, 'AP_eat_instr': 0.6554099418133656, 'AP_jump_instr': 0.6456856231732674, 'AP_lay_instr': 0.48510316603200604, 'AP_talk_on_phone_instr': 0.32534493753495364, 'AP_carry_obj': 0.31056494839355764, 'AP_throw_obj': 0.3796734753722468, 'AP_catch_obj': 0.3982827586677198, 'AP_cut_instr': 0.2928327529690103, 'AP_cut_obj': 0.5724285324807712, 'AP_run': 0.6341061427094135, 'AP_work_on_computer_instr': 0.5847372381774204, 'AP_ski_instr': 0.356957848582992, 'AP_surf_instr': 0.5642887969503578, 'AP_skateboard_instr': 0.6614059577255266, 'AP_smile': 0.7299096547302533, 'AP_drink_instr': 0.309137236519709, 'AP_kick_obj': 0.7904120602752323, 'AP_p

{'vcoco_val/vcoco': {'AP_hold_obj': 0.3989988749101249,
  'AP_stand': 0.7750321864972292,
  'AP_sit_instr': 0.423760818229507,
  'AP_ride_instr': 0.5329947832430317,
  'AP_walk': 0.5057719283058536,
  'AP_look_obj': 0.37318816204639255,
  'AP_hit_instr': 0.5900986007457361,
  'AP_hit_obj': 0.28914454117954863,
  'AP_eat_obj': 0.4578180872922468,
  'AP_eat_instr': 0.6554099418133656,
  'AP_jump_instr': 0.6456856231732674,
  'AP_lay_instr': 0.48510316603200604,
  'AP_talk_on_phone_instr': 0.32534493753495364,
  'AP_carry_obj': 0.31056494839355764,
  'AP_throw_obj': 0.3796734753722468,
  'AP_catch_obj': 0.3982827586677198,
  'AP_cut_instr': 0.2928327529690103,
  'AP_cut_obj': 0.5724285324807712,
  'AP_run': 0.6341061427094135,
  'AP_work_on_computer_instr': 0.5847372381774204,
  'AP_ski_instr': 0.356957848582992,
  'AP_surf_instr': 0.5642887969503578,
  'AP_skateboard_instr': 0.6614059577255266,
  'AP_smile': 0.7299096547302533,
  'AP_drink_instr': 0.309137236519709,
  'AP_kick_obj': 0.79